In [144]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()
with open("../login.json", "r") as j:
    misDatos = json.load(j)
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [152]:
nuevos = "C:/Users/jmartinez/Documents/final_metlife_base.xlsx"
Pacientes_Archivo = pd.read_excel(nuevos)
Pacientes_Archivo.head(5)

,Rut,Nombre,Sexo,Campaña,Holding,Empleador,Poliza,Fecha,Fecha_chilena,Fecha_chilena2,...,Hora,Dia_semana,Meses,Canal,Clínica,Nombre Empresa,Corredor,Pais,RUT_T,Index
0,174839182,Perez Cordova Catherine Fernanda,Mujer,MetLife,Concha Y Toro,Viña Concha Y Toro S.A.,340007542,2023-01-20T18:07:44.000+0000,2023-01-20 18:07:44,20-01-2023,...,18,viernes,Enero,Entrante - Llamada,Orientación Médica,Viña Concha Y Toro S.A.,Willis Towers Watson S.A. Corred. De Seg.,Chile,174839182,1
1,150235456,Katherine Evelyn Yong Leyton,Mujer,MetLife,Rol General SMU,Rendic Hnos. S.A.,340008362,2023-01-21T12:48:26.000+0000,2023-01-21 12:48:26,21-01-2023,...,12,sábado,Enero,Entrante - Llamada,Orientación Médica,Rendic Hnos. S.A.,THB Chile Corredores De Seguros S.A.,Chile,150235456,2
2,197272031,Rojo Collao Sebastian,Hombre,MetLife,Agrosuper,Agrícola Super Ltda.,340011786,2023-01-21T13:14:40.000+0000,2023-01-21 13:14:40,21-01-2023,...,13,sábado,Enero,Entrante - Chat Online,Orientación Médica,Agrícola Super Ltda.,Mercer Corredores De Seguros Ltda.,Chile,197272031,3
3,153308950,David Aaron Diaz Garcia,Hombre,MetLife,ACHS,Asociación Chilena De Seguridad,340007825,2023-01-21T18:19:00.000+0000,2023-01-21 18:19:00,21-01-2023,...,18,sábado,Enero,Saliente - Llamada,Orientación Médica,Asociación Chilena De Seguridad,Willis Towers Watson S.A. Corred. De Seg.,Chile,153308950,4
4,153308950,David Aaron Diaz Garcia,Hombre,MetLife,ACHS,Asociación Chilena De Seguridad,330000004,2023-01-21T18:19:00.000+0000,2023-01-21 18:19:00,21-01-2023,...,18,sábado,Enero,Saliente - Llamada,Orientación Médica,Fdo. De Salud Naci. De Los Trab. De ACHS,Mercer Corredores De Seguros Ltda.,Chile,153308950,5


In [153]:
df3 = pd.DataFrame()
Pacientes_Archivo.sort_values("Fecha_chilena", inplace=True)
Pacientes_Archivo.RUT_T.fillna(Pacientes_Archivo["Rut"], inplace=True)

Pacientes_Archivo_moviles = Pacientes_Archivo[(Pacientes_Archivo["Fecha"] >= "2023-02-01") & (Pacientes_Archivo["Fecha"] <= "2024-01-31")]
Pacientes_Archivo_moviles["llave"] = "Enero" + Pacientes_Archivo["Rut"].astype(str)

Pacientes_Archivo.head(5)
Pacientes_Archivo["llave"] = Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Rut"].astype(str)
#Pacientes_Archivo["llaveEnero"] = "Enero"+ Pacientes_Archivo["Rut_Titular_y"].astype(str)

Pacientes_Archivo["Familia_Mensual"] = Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Mes"].astype(str) + Pacientes_Archivo["RUT_T"].astype(str)
Pacientes_Archivo["rut_Unico_Mensual"] =  Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Mes"].astype(str) + Pacientes_Archivo["Rut"].astype(str)
Pacientes_Archivo["rut_Unico_Anual"] =  Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Rut"].astype(str)
Pacientes_Archivo["uso_titulares"] = np.where(Pacientes_Archivo["RUT_T"] == Pacientes_Archivo["Rut"], Pacientes_Archivo["Rut"].astype(str), 0)
Pacientes_Archivo["uso_titulares_Mes"] = np.where(Pacientes_Archivo["RUT_T"] == Pacientes_Archivo["Rut"], Pacientes_Archivo["Rut"].astype(str) + Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Mes"].astype(str) , 0)
Pacientes_Archivo["uso_titulares_Año"] = np.where(Pacientes_Archivo["RUT_T"] == Pacientes_Archivo["Rut"], Pacientes_Archivo["Rut"].astype(str) + Pacientes_Archivo["Año"].astype(str) , 0)
#Pacientes_Archivo["hora_Agenda"] = Pacientes_Archivo.Fecha_chilena.hour


Pacientes_Archivo.head()


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_8892\673111590.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pacientes_Archivo_moviles["llave"] = "Enero" + Pacientes_Archivo["Rut"].astype(str)


,Rut,Nombre,Sexo,Campaña,Holding,Empleador,Poliza,Fecha,Fecha_chilena,Fecha_chilena2,...,Pais,RUT_T,Index,llave,Familia_Mensual,rut_Unico_Mensual,rut_Unico_Anual,uso_titulares,uso_titulares_Mes,uso_titulares_Año
122256,156596981,Cifuentes Toloza Abraham Fernando,Masculino,MetLife,NaN,Multibien Servicio Bienestar Multibien - Holdi...,340018776,2022-01-02 20:58:36,2022-01-02,2022-01-02 20:58:36,...,Chile,156596981,122257,2022156596981,20221156596981,20221156596981,2022156596981,156596981,15659698120221,1565969812022
123525,129847514,Norambuena Alvaro Edson Benavides,Masculino,MetLife,NaN,Ingrammicro,340017722,2022-01-02 20:51:50,2022-01-02,2022-01-02 20:51:50,...,Chile,129847514,123526,2022129847514,20221129847514,20221129847514,2022129847514,129847514,12984751420221,1298475142022
122258,166711290,Del Pil Casanova Perez Soledad,Femenino,MetLife,NaN,La Araucana C.C.A.F.,340009888,2022-01-02 23:06:53,2022-01-02,2022-01-02 23:06:53,...,Chile,166711290,122259,2022166711290,20221166711290,20221166711290,2022166711290,166711290,16671129020221,1667112902022
122257,120823167,"ECHAGÜE, VERONICA PATRICIA VALENZUELA",Femenino,MetLife,NaN,Metlife,340018133,2022-01-02 21:08:45,2022-01-02,2022-01-02 21:08:45,...,Chile,120823167,122258,2022120823167,20221120823167,20221120823167,2022120823167,120823167,12082316720221,1208231672022
123524,94536049,Tabilo Martinez Carmen,Femenino,MetLife,NaN,La Araucana C.C.A.F.,340009888,2022-01-02 18:34:33,2022-01-02,2022-01-02 18:34:33,...,Chile,94536049,123525,202294536049,2022194536049,2022194536049,202294536049,94536049,9453604920221,945360492022


In [155]:
df3 = Pacientes_Archivo_moviles[[ "Index","llave"]]
df3 = df3.drop_duplicates(
    [
        "llave",
    ],
    keep="first",
)
df3["año_movil"] = 1 


In [136]:
Pacientes_Archivo.head()

,Codigo Ficha,Rut,Nombre,Sexo,Campaña,Holding,Empleador,Poliza_x,Fecha,Fecha_chilena,...,Relación_y,Index,llave,Familia_Mensual,rut_Unico_Mensual,rut_Unico_Anual,uso_titulares,uso_titulares_Mes,uso_titulares_Año,año_movil
0,FSA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,Titular,1,2022169397759,202211169397759,202211169397759,2022169397759,169397759,169397759202211,1693977592022,0.0
1,SA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,Titular,2,2022169397759,202211169397759,202211169397759,2022169397759,169397759,169397759202211,1693977592022,0.0
2,FSA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,Titular,3,2022106879508,202211106879508,202211106879508,2022106879508,106879508,106879508202211,1068795082022,0.0
3,SA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,Titular,4,2022106879508,202211106879508,202211106879508,2022106879508,106879508,106879508202211,1068795082022,0.0
4,FSA-0059,248270454,Aaron Antonio Aguayo Perez,Hombre,MetLife,Alsea,Starbucks Coffee Chile S.A.,340020238,2022-11-21T16:00:00.000+0000,2022-11-21 16:00:00,...,Carga,5,2022191841344,202211191841344,202211248270454,2022248270454,0,0,0,0.0


In [156]:
df3 = Pacientes_Archivo[[ "Index","llave"]]
df3 = df3.drop_duplicates(
    [
        "llave",
    ],
    keep="first",
)
df3["año_movil_Calenario"] = 1 
df3 = df3[[ "Index","año_movil_Calenario"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.año_movil_Calenario.fillna(0, inplace=True)


In [157]:
df3 = Pacientes_Archivo[[ "Index","Familia_Mensual"]]
df3 = df3.drop_duplicates(
    [
        "Familia_Mensual",
    ],
    keep="first",
)
df3["Familia_Mensual_CT"] = 1 
df3 = df3[[ "Index","Familia_Mensual_CT"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.Familia_Mensual_CT.fillna(0, inplace=True)
df3.head()

,Index,Familia_Mensual_CT
0,122257,1
1,123526,1
2,122259,1
3,122258,1
4,123525,1


In [158]:
#["rut Unico Mensual"]
df3 = Pacientes_Archivo[[ "Index","rut_Unico_Mensual"]]
df3 = df3.drop_duplicates(
    [
        "rut_Unico_Mensual",
    ],
    keep="first",
)
df3["rut_Unico_Mensual_CT"] = 1 
df3 = df3[[ "Index","rut_Unico_Mensual_CT"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.rut_Unico_Mensual_CT.fillna(0, inplace=True)
df3.head()

,Index,rut_Unico_Mensual_CT
0,122257,1
1,123526,1
2,122259,1
3,122258,1
4,123525,1


In [159]:
Pacientes_Archivo.head(5)


,Rut,Nombre,Sexo,Campaña,Holding,Empleador,Poliza,Fecha,Fecha_chilena,Fecha_chilena2,...,llave,Familia_Mensual,rut_Unico_Mensual,rut_Unico_Anual,uso_titulares,uso_titulares_Mes,uso_titulares_Año,año_movil_Calenario,Familia_Mensual_CT,rut_Unico_Mensual_CT
0,156596981,Cifuentes Toloza Abraham Fernando,Masculino,MetLife,NaN,Multibien Servicio Bienestar Multibien - Holdi...,340018776,2022-01-02 20:58:36,2022-01-02,2022-01-02 20:58:36,...,2022156596981,20221156596981,20221156596981,2022156596981,156596981,15659698120221,1565969812022,1.0,1.0,1.0
1,129847514,Norambuena Alvaro Edson Benavides,Masculino,MetLife,NaN,Ingrammicro,340017722,2022-01-02 20:51:50,2022-01-02,2022-01-02 20:51:50,...,2022129847514,20221129847514,20221129847514,2022129847514,129847514,12984751420221,1298475142022,1.0,1.0,1.0
2,166711290,Del Pil Casanova Perez Soledad,Femenino,MetLife,NaN,La Araucana C.C.A.F.,340009888,2022-01-02 23:06:53,2022-01-02,2022-01-02 23:06:53,...,2022166711290,20221166711290,20221166711290,2022166711290,166711290,16671129020221,1667112902022,1.0,1.0,1.0
3,120823167,"ECHAGÜE, VERONICA PATRICIA VALENZUELA",Femenino,MetLife,NaN,Metlife,340018133,2022-01-02 21:08:45,2022-01-02,2022-01-02 21:08:45,...,2022120823167,20221120823167,20221120823167,2022120823167,120823167,12082316720221,1208231672022,1.0,1.0,1.0
4,94536049,Tabilo Martinez Carmen,Femenino,MetLife,NaN,La Araucana C.C.A.F.,340009888,2022-01-02 18:34:33,2022-01-02,2022-01-02 18:34:33,...,202294536049,2022194536049,2022194536049,202294536049,94536049,9453604920221,945360492022,1.0,1.0,1.0


In [160]:
from pivottablejs import pivot_ui

In [161]:
Pacientes_Archivo.to_excel("C:/Users/jmartinez/Documents/reportes_2111.xlsx", index=False)

In [ ]:


#Pacientes_Archivo.to_excel("C:/Users/jmartinez/Documents/reportes_2111.xlsx", index=False)
pivot_ui(Pacientes_Archivo)